# Segmenting and clustering Neighborhoods in Toronto 


## Peer graded assignment within Data science Capstone course

***

### Scrape Wikipedia for data on Toronto Neighborhoods and make a DataFrame

- Install Beautiful soup

In [10]:
!pip install beautifulsoup4

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


- import necessary packages

In [11]:
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import requests

- set URL and use requests package to get response object then declare Beautiful soup object and show it using pretiffy method

In [12]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

html_text = requests.get(url).text

soup = BeautifulSoup(html_text, 'lxml')
print(soup.prettify()[5000:15000])


org" rel="dns-prefetch"/>
  <link href="//meta.wikimedia.org" rel="dns-prefetch"/>
 </head>
 <body class="mediawiki ltr sitedir-ltr mw-hide-empty-elt ns-0 ns-subject page-List_of_postal_codes_of_Canada_M rootpage-List_of_postal_codes_of_Canada_M skin-vector action-view skin-vector-legacy">
  <div class="noprint" id="mw-page-base">
  </div>
  <div class="noprint" id="mw-head-base">
  </div>
  <div class="mw-body" id="content" role="main">
   <a id="top">
   </a>
   <div id="siteNotice">
    <!-- CentralNotice -->
   </div>
   <div class="mw-indicators">
    <div class="mw-indicator" id="mw-indicator-pp-default">
     <a href="/wiki/Wikipedia:Protection_policy#semi" title="This article is semi-protected until June 20, 2021 at 02:13 UTC.">
      <img alt="Page semi-protected" data-file-height="512" data-file-width="512" decoding="async" height="20" src="//upload.wikimedia.org/wikipedia/en/thumb/1/1b/Semi-protection-shackle.svg/20px-Semi-protection-shackle.svg.png" srcset="//upload.wikimed

-  use find() method on soup object to get the table. 
- loop through each row under a "td" tag
- check to see if the "span" tag is not assigned and skip it, if so
- using temp_dict, set value of PostalCode key to the text up to the 3rd element of the p tag, which should be the postal code, then set the value of Bourough to the first element in the list after using split() on the text in the span tag, which should be the borough, then use the second element in the span.text.split() list to assign to neighborhood and get rid of the extra characters and white space
- add this temp_dict to the list table_data, then after the loop pass the list into a pandas dataframe
- show the first 5 rows

In [13]:

table = soup.find("table")
df_data = []
for row in table.findAll("td"):
    temp_dict = {}
    if row.span.text == "Not assigned":
        pass
    else:
        temp_dict['PostalCode'] = row.p.text[:3]
        temp_dict['Borough'] = row.span.text.split('(')[0]
        temp_dict['Neighborhood'] = row.span.text.split('(')[1]
        temp_dict['Neighborhood'] = temp_dict['Neighborhood'].strip(')').strip(' ')
        temp_dict['Neighborhood'] = temp_dict['Neighborhood'].replace('/',',').replace(')', '')
        df_data.append(temp_dict)
        
df = pd.DataFrame(df_data, columns = ["PostalCode", "Borough", "Neighborhood"])
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


- show the shape of the df

In [14]:
print("\nShape: ", df.shape)


Shape:  (103, 3)


***

### Use Geocoder or csv file to add latitude and longitude columns to the DataFrame

In [15]:
!pip install geocoder

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


- geocoder not working, will use csv file in next cell.

import geocoder

lat_values = []
long_values = []
for postal_code in df['PostalCode']:
    lat_long_cords = None

    while lat_long_cords is None:
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_long_cords = g.latlng
    latitude = lat_long_cords[0]
    longitude = lat_long_cords[1]
    lat_values.append(latitude)
    long_values.append(longitude)
df['latitude'] = lat_values
df['longitude'] = long_values
df.head()

- Below i will read in the csv data into a df from IBM watson storage (first cell)
- Then sort both frames to make sure postal codes line up (second cell)
- Then add the latitude and longitude values from the csv df to the main df

In [18]:
# The code was removed by Watson Studio for sharing.

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [19]:
geo_data_df = df_data_1

geo_data_df.sort_values(by='Postal Code', inplace=True)
df.sort_values(by='PostalCode', inplace=True)

print(geo_data_df.head())
print("\n", df.head())

df['latitude'] = geo_data_df['Latitude']
df['longitude'] = geo_data_df['Longitude']

  Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476

    PostalCode      Borough                              Neighborhood
6         M1B  Scarborough                           Malvern , Rouge
12        M1C  Scarborough  Rouge Hill , Port Union , Highland Creek
18        M1E  Scarborough       Guildwood , Morningside , West Hill
22        M1G  Scarborough                                    Woburn
26        M1H  Scarborough                                 Cedarbrae


- Re-sort by the index and show the shape and head of the new df

In [20]:

print("\ndf shape: ", df.shape)
df.sort_index(inplace=True)
df.head()


df shape:  (103, 5)


,PostalCode,Borough,Neighborhood,latitude,longitude
0,M3A,North York,Parkwoods,43.806686,-79.194353
1,M4A,North York,Victoria Village,43.784535,-79.160497
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.763573,-79.188711
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.770992,-79.216917
4,M7A,Queen's Park,Ontario Provincial Government,43.773136,-79.239476


***

### Explore and Cluster the data

In [21]:
print("The dataframe has {} postal codes and {} Neighborhoods".format(
    len(df['PostalCode'].unique()), 
    len(df['Neighborhood'].unique())
        )
     )

The dataframe has 103 postal codes and 103 Neighborhoods


- import folium and geopy packages
- Then get lat and long of Toronto from geopy

In [24]:
!pip install geopy
!pip install folium

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 94 kB 5.0 MB/s  eta 0:00:01


In [25]:
import folium
from geopy.geocoders import Nominatim

In [26]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent='tor_explorer')
location = geolocator.geocode(address)
latitude_tor = location.latitude
longitude_tor = location.longitude
print("The geographical coordinates of Toronto Ontario are {}, {}".format(latitude_tor, longitude_tor))

The geographical coordinates of Toronto Ontario are 43.6534817, -79.3839347


- Create a map of Toronto and superimpose the df coordinates to map out the neighoborhoods

In [27]:
map_toronto = folium.Map(location=[latitude_tor, longitude_tor], zoom_start=12)

for lat, lng, borough, neigh in zip(df['latitude'], df['longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neigh, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#00959f',
        fill_opacity=0.6,
        parse_html=False
        ).add_to(map_toronto)
    
map_toronto